<a href="https://colab.research.google.com/github/avyukthinna/ML_Lab/blob/main/1BM22CS060_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math
import numpy as np

# Gini Impurity for binary classification
def gini_index(groups, classes):
    n_instances = sum([len(group) for group in groups])
    gini = 0.0

    for group in groups:
        size = len(group)
        if size == 0:
            continue

        score = 0.0
        labels = [row[-1] for row in group]
        for class_val in classes:
            p = labels.count(class_val) / size
            score += p * p
        gini += (1 - score) * (size / n_instances)
    return gini

# Split dataset by feature and value
def test_split(index, value, dataset):
    left, right = [], []
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Choose best split
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    best_index, best_value, best_score, best_groups = None, None, float('inf'), None
    features = random.sample(range(len(dataset[0]) - 1), n_features)

    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < best_score:
                best_index, best_value, best_score, best_groups = index, row[index], gini, groups
    return {'index': best_index, 'value': best_value, 'groups': best_groups}

# Terminal node
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Recursive splitting
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])

    # If no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return

    # Max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return

    # Left split
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth + 1)

    # Right split
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth + 1)

# Build tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

# Make prediction with a tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

# Create a random subsample (bootstrap)
def subsample(dataset, ratio):
    sample = []
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = random.randrange(len(dataset))
        sample.append(dataset[index])
    return sample

# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_ratio, n_trees, n_features):
    trees = []
    for _ in range(n_trees):
        sample = subsample(train, sample_ratio)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)

    predictions = [bagging_predict(trees, row) for row in test]
    return predictions

# Voting prediction
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

# Accuracy
def accuracy_metric(actual, predicted):
    correct = sum([1 for i in range(len(actual)) if actual[i] == predicted[i]])
    return correct / len(actual) * 100.0

# ==== Dataset ====
dataset = [
    [25, 55, 1],
    [30, 60, 1],
    [22, 58, 1],
    [35, 85, 0],
    [40, 90, 0],
    [45, 88, 0],
    [25, 60, 1],
    [31, 70, 0],
    [19, 50, 1],
    [40, 80, 0]
]

# Test with same data (since we're demonstrating)
train_set = dataset
test_set = dataset
actual_labels = [row[-1] for row in test_set]

# Run Random Forest
predicted = random_forest(train_set, test_set, max_depth=5, min_size=1, sample_ratio=1.0, n_trees=5, n_features=1)

# Output
print("Actual   :", actual_labels)
print("Predicted:", predicted)
print("Accuracy :", accuracy_metric(actual_labels, predicted), "%")


Actual   : [1, 1, 1, 0, 0, 0, 1, 0, 1, 0]
Predicted: [1, 1, 1, 0, 0, 0, 1, 0, 1, 0]
Accuracy : 100.0 %
